In [1]:
import numpy as np
import pickle
import os

# 1. Load toàn bộ tập train
train_data_path  = "./train_data.npy"
train_labels_path = "./train_labels.pkl"

data = np.load(train_data_path)            # dtype: float32, shape: (N, T, P, J, C) ?
with open(train_labels_path, "rb") as f:
    labels = pickle.load(f)

print("Train data shape:", data.shape)
# Ví dụ: (1600, 30, 3, 17, 3)

print("Train labels shape:", labels.shape)
# Ví dụ: (1600,)

# 2. Kiểm tra sample đơn lẻ (để chắc chắn thứ tự axes)
sample = data[0]
print("\nSample[0] shape:", sample.shape)
# => (T, P, J, C)

# 3. In thử một vài giá trị min/max để xem scale
print("Sample[0] min, max per channel:", sample[...,0].min(), sample[...,0].max(),
      sample[...,1].min(), sample[...,1].max(),
      sample[...,2].min(), sample[...,2].max())

# 4. Nếu thấy axes sai (ví dụ shape=(T, P, C, J) hoặc (T, J, P, C)), hãy thử hoán permute
#    và xem shape mới:
# permuted = sample.transpose(0, 2, 3, 1)
# print("After transpose(0,2,3,1):", permuted.shape)


Train data shape: (1600, 16, 5, 17, 3)
Train labels shape: (1600,)

Sample[0] shape: (16, 5, 17, 3)
Sample[0] min, max per channel: 0.0 0.9567818 0.0 1.0 0.0 0.99173987
